Experiment on filtered labels with k-mers instead of chars. 
5-fold validation.

In [2]:
from gensim.models import Word2Vec
import annoy # fast approximate KNN
import numpy as np

In [3]:
!head ./UP000002311.goByGeneC

gene seq annot
"P32317" "MIALKPNAVRTFRQVQHCSFRICRYQSTKSNKCLTPLQEYDRLVKLGKLRDDTYQRGIISSLGDLYDSLVKYVPPVVKTPNAVDQVGGWLNGLKSVFSRGKPKNIGAYVDVSKIGNSIPRGVYLYGDVGCGKTMLMDLFYTTIPNHLTKKRIHFHQFMQYVHKRSHEIVREQNLKELGDAKGKEIDTVPFLAAEIANNSHVLCFDEFQVTDVADAMILRRLMTALLSDDYGVVLFATSNRHPDELYINGVQRQSFIPCIELIKHRTKVIFLNSPTDYRKIPRPVSSVYYFPSDTSIKYASKECKTRRETHIKEWYNYFAQASHTDDSTDSHTVHKTFYDYPLTIWGREFKVPKCTPPRVAQFTFKQLCGEPLAAGDYLTLAKNFEAFIVTDIPYLSIYVRDEVRRFITFLDAVYDSGGKLATTGAADFSSLFVEPEQILNDFELRPTTKEPDSVDTGMVDEMVEKHGFSKEIAKKSQMFALDEERFAFARALSRLSQMSSTDWVTKPTY" "GO:0005575, GO:0005622, GO:0005623, GO:0005737, GO:0005739, GO:0043226, GO:0043227, GO:0043229, GO:0043231, GO:0044424, GO:0044444, GO:0044464"
"P87275" "MIEEKKELKKRRVLQMARFYGAAAFTLITMRLISRAIKVRKYVPSIFQQNYKLPPFSQRNEAMSALTYASAASIGTFSTLIFGFCWALDISTAREFVFKTREFMSLPQALETDTSMDEETSKLTKQLQDLLSSENNK" "GO:0005575, GO:0005622, GO:0005623, GO:0005737, GO:0005739, GO:0043226, GO:0043227, GO:0043229, GO:0043231, GO:0044424, GO:0044444, GO:0044464"
"P39010" "MVNELENVPRASTLT

In [3]:
good_labels = set(['GO:0005634',# nucleus
'GO:0005576',# extracellular
'GO:0005829',# cytosol
'GO:0005856',# cytoskeleton
'GO:0005739',# mitochondrion
'GO:0005886',# cell_membrane
'GO:0005783',# endoplasmic_reticulum
'GO:0005794',# golgi_apparatus
'GO:0005773',# vacuole
'GO:0005777',# peroxysome
              ])

In [5]:
X = [] # sequence strings
Y = [] # list of labels for each sequence
all_y = set() # all the labels collected from the dataset
with open('./UP000002311.goByGeneC') as inp:
    inp.readline() # header
    for line in inp:
        tokens = map(lambda x: x.strip('"'), line.split(' ',2))
        
        object_labels = [t.strip().strip('"') for t in tokens[2].split(',') if len(t.strip())>0 and t.strip]
        object_labels = [y for y in object_labels if y in good_labels]
        if len(object_labels)>=1:
            X.append(tokens[1])
            Y.append(object_labels)
            all_y = all_y.union(set(Y[-1]))

In [6]:
print 'example of X', X[:3]
print 'example of Y', Y[:3]
print 'example of set of labels', list(all_y)[:3]
print 'total sequences', len(X)
print 'total labels', len(all_y)

example of X ['MIALKPNAVRTFRQVQHCSFRICRYQSTKSNKCLTPLQEYDRLVKLGKLRDDTYQRGIISSLGDLYDSLVKYVPPVVKTPNAVDQVGGWLNGLKSVFSRGKPKNIGAYVDVSKIGNSIPRGVYLYGDVGCGKTMLMDLFYTTIPNHLTKKRIHFHQFMQYVHKRSHEIVREQNLKELGDAKGKEIDTVPFLAAEIANNSHVLCFDEFQVTDVADAMILRRLMTALLSDDYGVVLFATSNRHPDELYINGVQRQSFIPCIELIKHRTKVIFLNSPTDYRKIPRPVSSVYYFPSDTSIKYASKECKTRRETHIKEWYNYFAQASHTDDSTDSHTVHKTFYDYPLTIWGREFKVPKCTPPRVAQFTFKQLCGEPLAAGDYLTLAKNFEAFIVTDIPYLSIYVRDEVRRFITFLDAVYDSGGKLATTGAADFSSLFVEPEQILNDFELRPTTKEPDSVDTGMVDEMVEKHGFSKEIAKKSQMFALDEERFAFARALSRLSQMSSTDWVTKPTY', 'MIEEKKELKKRRVLQMARFYGAAAFTLITMRLISRAIKVRKYVPSIFQQNYKLPPFSQRNEAMSALTYASAASIGTFSTLIFGFCWALDISTAREFVFKTREFMSLPQALETDTSMDEETSKLTKQLQDLLSSENNK', 'MVNELENVPRASTLTNEEQTVDPSNNDSQEDISLGDSNEITSLASLKAIRSGNEEESGNEQVNHNDEAEEDPLLTRYHTACQRGDLATVKEMIHGKLLEVNNDGDSTEHITGLHWASINNRLSVVDFLVSQGADVNARAGALHATPLHWAARYGYVYIVDFLLKHGADPTMTDDQGFNLLHLSVNSSNIMLVLYVLFNVVSKGLLDIDCRDPKGRTSLLWAAYQGDSLTVAELLKFGASIKIADTEGFTPLHWGTVKGQPHVLKYLIQDGADFFQKTDTGKDCFAIAQEMNTVYSLREALTHSGFDYHGYPIKKWFKKSQHAKLVTFITPF

In [7]:
def cv_iter(w2v_params, k=3, overlap=False):    
    def kmer(s):
        if overlap:
            step = 1
        else:
            step = k
        
        result = [s[i:i+k] for i in xrange(0, len(s)-k, step)]
        if len(result) == 0:
            return s 
        return result
        
    """
    one ieration of cross-validation
    w2v_params is a dictionary with parameters of gensim.Word2Vec
    """
    indices = range(len(X))
    np.random.shuffle(indices)

    Xtrain = np.array(X)[indices[:len(indices)/2]]
    Ytrain = np.array(Y)[indices[:len(indices)/2]]
    Xtest = np.array(X)[indices[len(indices)/2:]]
    Ytest = np.array(Y)[indices[len(indices)/2:]]
    # split all the strings into lists for word2vec traing
    Xtrain = [kmer(x) for x in Xtrain]
    Xtest = [kmer(x) for x in Xtest]
    model = Word2Vec(Xtrain, **w2v_params)
    index = annoy.AnnoyIndex(w2v_params.get('size', 100)) # default value is 100 for gensim.Word2Vec
    for i in range(len(Xtrain)):
        v= np.mean([model[c] for c in Xtrain[i] if c in model], axis=0) # centroid
        if np.isnan(np.mean(v)):
            continue
        index.add_item(i, v)
    index.build(100) # 100 is a number of trees in index, we can alibrate this value
    predicted = []
    for i in range(len(Xtest)):
        v= np.mean([model[c] for c in Xtest[i]  if c in model], axis=0)
        if np.isnan(np.mean(v)):
            id = -1
        else:
            id = index.get_nns_by_vector(v,1)[0] #  finding 1 nearest neighbour
        predicted.append(id)
    tp = 0
    fp = 0
    true_label_num = 0
    for i in xrange(len(Xtest)):
        true = set(Ytest[i])
        true_label_num+=len(true)
        pred = set(Ytrain[predicted[i]])
        tp += len(true&pred)
        fp += len(pred) - len(true&pred)

    prec=tp*1.0/(tp+fp)
    rec = tp*1.0/true_label_num
    return prec, rec,  2 * prec * rec/ (prec+rec)
    

In [8]:
def cv(w2v_params = {}, k=3, overlap=False, cv_num = 5 ):
    """
    Cross-validation function.
    Returns list with precision, recall and F1 values
    """
    precs, recs, fs = [],[],[]
    for _ in range(cv_num):
        p,r,f = cv_iter(w2v_params , k=k, overlap=overlap)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    return precs, recs, fs

In [9]:
# simple test, checking everything is correct. The results should be the same as without kmers
p,r,f = cv(k=1)
np.mean(p), np.mean(r), np.mean(f)

(0.40283925227577555, 0.40872368758477784, 0.40575593246048464)

In [10]:
p,r,f = cv(k=2)
np.mean(p), np.mean(r), np.mean(f)

(0.3817316749082145, 0.3826022563171871, 0.38215667230051475)

In [ ]:
p,r,f = cv(k=3)
np.mean(p), np.mean(r), np.mean(f)

In [12]:
p,r,f = cv(k=10)
np.mean(p), np.mean(r), np.mean(f)

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(0.24736739817214098, 0.24045346291748276, 0.23397273419251144)

In [13]:
p,r,f = cv(k=2, overlap=True)
np.mean(p), np.mean(r), np.mean(f)

(0.4161790991473529, 0.42664144439367463, 0.42128835478192006)

In [14]:
p,r,f = cv(k=3, overlap=True)
np.mean(p), np.mean(r), np.mean(f)

(0.4011508585690322, 0.4120144530122272, 0.40649199810169795)

In [10]:
# Skip-Gramms 
p,r,f = cv(w2v_params={"sg":1}, k=1, overlap=True)
np.mean(p), np.mean(r), np.mean(f)

(0.4086118885572902, 0.417190077251247, 0.41283748330960834)